# Checking SNP and haplotype genotyping error mismatch

I'm going a little crazy trying to understand how there could be such a discrepancy in genotyping error between the SNP and haplotype Genepop files, so I thought it was worth meticulously double-checking to make sure the error isn't me. It's usually me.

## First: SNP genotyping error
First, I'll take a look at the SNP Genepop file. It looks like this:

![image](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/snp_gp.png?raw=true)

In [59]:
cd /mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2

/mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2


I copied and pasted just the rows for the individuals with replicates, into this file 'cragigrun1_sampreps_snps_gen.txt', and it looks like this:

![image](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/pic_samps_for_snp_gen_error.png?raw=true)

with the rows extracted for samples with replicates, and also with the locus names at the bottom.

**HOWEVER** To double-check, I'm going to use a test file, which has only the first 11 SNPs, and looks like this:
![image](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/test_snps_reps.png?raw=true)
I'm using a test file so that I can introduce a known amount of error, and see how well it does.

I introduced the following errors:
* FG101: 1 total mismatch, 1 partial mismatch
* FG102: 2 total mismatches
* FG103: no mismatches
* FG104: 1 total mismatch, 2 partial mismatches

In [60]:
input = open("test_replicates.txt") # read in file
lines = input.readlines()
biglist = []
for line in lines:
    linelist = line.strip().split()
    biglist.append(linelist) # prepare a list to make an array

# check list of lists is what you expect
print biglist[0][0:9]

['FG101_A,', '0000', '0000', '0000', '0303', '0101', '0202', '0202', '0101']


In [75]:
headerline = lines[8]
snp_list = headerline.strip().split(",")

print snp_list[0:11]

['5_75', '5_84', '5_137', '6_16', '6_23', '6_24', '6_40', '6_74', '6_97', '6_112', '6_126']


In [62]:
num_snps = len(snp_list) # iterate over number of tags
print "You have " + str(num_snps) + " SNPs."

You have 11 SNPs.


In [63]:
import numpy as np
myarray = np.array(biglist) # turn the file into an array

Assign rows of replicates to make small arrays w two rows each, for each replicated individual.

In [64]:
FG101 = np.array(biglist[0:2])
FG102 = np.array(biglist[2:4])
FG103 = np.array(biglist[4:6])
FG104 = np.array(biglist[6:8])

Double-check row assignments...

In [65]:
FG101[0:2,0:7]


array([['FG101_A,', '0000', '0000', '0000', '0303', '0101', '0202'],
       ['FG101_B,', '0000', '0000', '0000', '0303', '0101', '0202']], 
      dtype='|S8')

In [66]:
# put arrays in list
list_of_arrays = []
list_of_arrays.append(FG101)
list_of_arrays.append(FG102)
list_of_arrays.append(FG103)
list_of_arrays.append(FG104)

In [67]:
# initiate counts and lists to track genotyping error
match_genotypes_count = 0
match_genotypes_tags = []
partmismatch_genotypes_count = 0
partmismatch_genotypes_tags = []
totalmismatch_genotypes_count = 0
totalmismatch_genotypes_tags = []
nodata_mismatch_count = 0
nodata_mismatch_tags = []

In [81]:
# sort genotype match/mismatch by sample
bysam_match_tags = []
bysam_partmismatch_tags = []
bysam_totalmismatch_tags = []
bysam_missing_tags = []

timescount = 1
for array in list_of_arrays:
    sam_match_genotypes_count = 0
    sam_match_genotypes_tags = []
    sam_partmismatch_genotypes_count = 0
    sam_partmismatch_genotypes_tags = []
    sam_totalmismatch_genotypes_count = 0
    sam_totalmismatch_genotypes_tags = []
    sam_nodata_mismatch_count = 0
    sam_nodata_mismatch_tags = []
    for i in range(1,num_snps+1):
        A_gen = array[0][i] # get genotypes
        B_gen = array[1][i]
        A_1 = A_gen[0:2] # pull out alleles
        A_2 = A_gen[2:4]
        B_1 = B_gen[0:2]
        B_2 = B_gen[2:4]
        A_list = [] # store alleles in lists
        A_list.append(A_1)
        A_list.append(A_2)
        B_list = []
        B_list.append(B_1)
        B_list.append(B_2)
        first_test_string = B_1 + B_2
        second_test_string = B_2 + B_1
        match_options = []
        match_options.append(first_test_string)
        match_options.append(second_test_string)
        if (A_gen == "0000" or B_gen == "0000") or (A_gen == "0000" and B_gen == "0000"):
            nodata_mismatch_count += 1
            nodata_mismatch_tags.append(snp_list[i-1])
            sam_nodata_mismatch_count +=1
            sam_nodata_mismatch_tags.append(snp_list[i-1])
        elif A_gen in match_options: # if perfect match
            match_genotypes_count += 1
            match_genotypes_tags.append(snp_list[i-1])
            sam_match_genotypes_count += 1
            sam_match_genotypes_tags.append(snp_list[i-1])
        elif A_1 in B_list or A_2 in B_list: # if one allele is a match
            partmismatch_genotypes_count += 1
            partmismatch_genotypes_tags.append(snp_list[i-1])
            sam_partmismatch_genotypes_count += 1
            sam_partmismatch_genotypes_tags.append(snp_list[i-1])
        elif A_1 not in B_list or A_2 not in B_list: # if not a match at all
            totalmismatch_genotypes_count += 1
            totalmismatch_genotypes_tags.append(snp_list[i-1])
            sam_totalmismatch_genotypes_count += 1
            sam_totalmismatch_genotypes_tags.append(snp_list[i-1])
        else:
            print "Your loop may not be fully working." 
            print "You thought there were three possible options, but there were more than three. Check!"
        
    bysam_missing_tags.append(sam_nodata_mismatch_tags)
    bysam_match_tags.append(sam_match_genotypes_tags)
    bysam_partmismatch_tags.append(sam_partmismatch_genotypes_tags)
    bysam_totalmismatch_tags.append(sam_totalmismatch_genotypes_tags)

    print "\nSample number " + str(timescount) + " :"
    print "Genotypes with perfect match: " + str(sam_match_genotypes_count) + " : "
    print "Genotypes with partial match: " + str(sam_partmismatch_genotypes_count) + " : "
    print "Genotypes with total mismatch: " + str(sam_totalmismatch_genotypes_count) + " : "
    print "Genotypes with mismatch due to missing data: " + str(sam_nodata_mismatch_count) + " : "
        
    timescount +=1


Sample number 1 :
Genotypes with perfect match: 6 : 
Genotypes with partial match: 1 : 
Genotypes with total mismatch: 1 : 
Genotypes with mismatch due to missing data: 3 : 

Sample number 2 :
Genotypes with perfect match: 9 : 
Genotypes with partial match: 0 : 
Genotypes with total mismatch: 2 : 
Genotypes with mismatch due to missing data: 0 : 

Sample number 3 :
Genotypes with perfect match: 11 : 
Genotypes with partial match: 0 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 0 : 

Sample number 4 :
Genotypes with perfect match: 8 : 
Genotypes with partial match: 2 : 
Genotypes with total mismatch: 1 : 
Genotypes with mismatch due to missing data: 0 : 


In [82]:
from __future__ import division

total_excl_mv = match_genotypes_count + partmismatch_genotypes_count + totalmismatch_genotypes_count
print "Total genotypes excluding those with missing data : " + str(total_excl_mv)
print "\nTotal genotypes with perfect match: " + str(match_genotypes_count) + " : " + str((float(match_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with partial match: " + str(partmismatch_genotypes_count) + " : " + str((float(partmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with total mismatch: " + str(totalmismatch_genotypes_count) + " : " + str((float(totalmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "\nTotal genotypes with mismatch due to missing data: " + str(nodata_mismatch_count)

Total genotypes excluding those with missing data : 233

Total genotypes with perfect match: 191 : 81.97%
Total genotypes with partial match: 17 : 7.296%
Total genotypes with total mismatch: 25 : 10.72%

Total genotypes with mismatch due to missing data: 30


**So, does this match what is known?** Yes it does!

Again, what is known about the test file:
* FG101: 1 total mismatch, 1 partial mismatch
* FG102: 2 total mismatches
* FG103: no mismatches
* FG104: 1 total mismatch, 2 partial mismatches

I did find a small indexing problem, and solved it in the code, but unfortunately I don't think it will change the outcome at all for the real file. Well, I'll find out here!

## Now with the actual file!

In [83]:
input = open("cragigrun1_sampreps_snps_gen.txt") # read in file
lines = input.readlines()
biglist = []
for line in lines:
    linelist = line.strip().split()
    biglist.append(linelist) # prepare a list to make an array

# check list of lists is what you expect
print biglist[0][0:9]

['FG101_A,', '0000', '0000', '0000', '0303', '0101', '0202', '0202', '0101']


In [85]:
headerline = lines[8]
snp_list = headerline.strip().split(",")

print snp_list[0:19]

['5_75', '5_84', '5_137', '6_16', '6_23', '6_24', '6_40', '6_74', '6_97', '6_112', '6_126', '6_129', '13_47', '13_51', '13_72', '13_81', '13_83', '13_86', '13_93']


In [86]:
num_snps = len(snp_list) # iterate over number of tags
print "You have " + str(num_snps) + " SNPs."

You have 82265 SNPs.


In [87]:
import numpy as np
myarray = np.array(biglist) # turn the file into an array

In [88]:
FG101 = np.array(biglist[0:2])
FG102 = np.array(biglist[2:4])
FG103 = np.array(biglist[4:6])
FG104 = np.array(biglist[6:8])

In [89]:
FG101[0:2,0:7]

array([['FG101_A,', '0000', '0000', '0000', '0303', '0101', '0202'],
       ['FG101_B,', '0000', '0000', '0000', '0303', '0101', '0202']], 
      dtype='|S8')

In [90]:
# put arrays in list
list_of_arrays = []
list_of_arrays.append(FG101)
list_of_arrays.append(FG102)
list_of_arrays.append(FG103)
list_of_arrays.append(FG104)

In [91]:
# initiate counts and lists to track genotyping error
match_genotypes_count = 0
match_genotypes_tags = []
partmismatch_genotypes_count = 0
partmismatch_genotypes_tags = []
totalmismatch_genotypes_count = 0
totalmismatch_genotypes_tags = []
nodata_mismatch_count = 0
nodata_mismatch_tags = []

In [92]:
# sort genotype match/mismatch by sample
bysam_match_tags = []
bysam_partmismatch_tags = []
bysam_totalmismatch_tags = []
bysam_missing_tags = []

timescount = 1
for array in list_of_arrays:
    sam_match_genotypes_count = 0
    sam_match_genotypes_tags = []
    sam_partmismatch_genotypes_count = 0
    sam_partmismatch_genotypes_tags = []
    sam_totalmismatch_genotypes_count = 0
    sam_totalmismatch_genotypes_tags = []
    sam_nodata_mismatch_count = 0
    sam_nodata_mismatch_tags = []
    for i in range(1,num_snps+1):
        A_gen = array[0][i] # get genotypes
        B_gen = array[1][i]
        A_1 = A_gen[0:2] # pull out alleles
        A_2 = A_gen[2:4]
        B_1 = B_gen[0:2]
        B_2 = B_gen[2:4]
        A_list = [] # store alleles in lists
        A_list.append(A_1)
        A_list.append(A_2)
        B_list = []
        B_list.append(B_1)
        B_list.append(B_2)
        first_test_string = B_1 + B_2
        second_test_string = B_2 + B_1
        match_options = []
        match_options.append(first_test_string)
        match_options.append(second_test_string)
        if (A_gen == "0000" or B_gen == "0000") or (A_gen == "0000" and B_gen == "0000"):
            nodata_mismatch_count += 1
            nodata_mismatch_tags.append(snp_list[i-1])
            sam_nodata_mismatch_count +=1
            sam_nodata_mismatch_tags.append(snp_list[i-1])
        elif A_gen in match_options: # if perfect match
            match_genotypes_count += 1
            match_genotypes_tags.append(snp_list[i-1])
            sam_match_genotypes_count += 1
            sam_match_genotypes_tags.append(snp_list[i-1])
        elif A_1 in B_list or A_2 in B_list: # if one allele is a match
            partmismatch_genotypes_count += 1
            partmismatch_genotypes_tags.append(snp_list[i-1])
            sam_partmismatch_genotypes_count += 1
            sam_partmismatch_genotypes_tags.append(snp_list[i-1])
        elif A_1 not in B_list or A_2 not in B_list: # if not a match at all
            totalmismatch_genotypes_count += 1
            totalmismatch_genotypes_tags.append(snp_list[i-1])
            sam_totalmismatch_genotypes_count += 1
            sam_totalmismatch_genotypes_tags.append(snp_list[i-1])
        else:
            print "Your loop may not be fully working." 
            print "You thought there were three possible options, but there were more than three. Check!"
        
    bysam_missing_tags.append(sam_nodata_mismatch_tags)
    bysam_match_tags.append(sam_match_genotypes_tags)
    bysam_partmismatch_tags.append(sam_partmismatch_genotypes_tags)
    bysam_totalmismatch_tags.append(sam_totalmismatch_genotypes_tags)

    print "\nSample number " + str(timescount) + " :"
    print "Genotypes with perfect match: " + str(sam_match_genotypes_count) + " : "
    print "Genotypes with partial match: " + str(sam_partmismatch_genotypes_count) + " : "
    print "Genotypes with total mismatch: " + str(sam_totalmismatch_genotypes_count) + " : "
    print "Genotypes with mismatch due to missing data: " + str(sam_nodata_mismatch_count) + " : "
        
    timescount +=1


Sample number 1 :
Genotypes with perfect match: 79256 : 
Genotypes with partial match: 246 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 2763 : 

Sample number 2 :
Genotypes with perfect match: 79611 : 
Genotypes with partial match: 275 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 2379 : 

Sample number 3 :
Genotypes with perfect match: 78651 : 
Genotypes with partial match: 362 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 3252 : 

Sample number 4 :
Genotypes with perfect match: 78819 : 
Genotypes with partial match: 293 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 3153 : 


In [93]:
from __future__ import division

total_excl_mv = match_genotypes_count + partmismatch_genotypes_count + totalmismatch_genotypes_count
print "Total genotypes excluding those with missing data : " + str(total_excl_mv)
print "\nTotal genotypes with perfect match: " + str(match_genotypes_count) + " : " + str((float(match_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with partial match: " + str(partmismatch_genotypes_count) + " : " + str((float(partmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with total mismatch: " + str(totalmismatch_genotypes_count) + " : " + str((float(totalmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "\nTotal genotypes with mismatch due to missing data: " + str(nodata_mismatch_count)

Total genotypes excluding those with missing data : 317513

Total genotypes with perfect match: 316337 : 99.62%
Total genotypes with partial match: 1176 : 0.370%
Total genotypes with total mismatch: 0 : 0.0%

Total genotypes with mismatch due to missing data: 11547


Looks like the genotyping error wasn't affected by my minor code fixes

## Second: haplotype genotyping error

### haplotypes -> hap Genepop file check

I wrote a script that takes the haplotypes file from ``populations`` and writes a Genepop. I spot checked it, but it could be the source of the problem because people make mistakes (and computers don't!). So double-checking that it worked using a test haplotype file.

I made a test haplotypes file previously, it looks like this:
![image](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Notebooks/images_for_notebooks/test_hap.png?raw=true)

So... eyeballing this test haplotypes file, I see:
* Locus 5: 4 alleles
* Locus 6: 2 alleles
* Locus 13: 2 alleles
* Locus 16: 3 alleles
* Locus 20:

In [94]:
cd /mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts

/mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts


In [ ]:
!python parse_hapfile_to_tagGenePop.py \
-f ../../../WorkingFolder/Stacks_2/test_hap.tsv \
-p ../../../WorkingFolder/popmap_cragigrun1.txt \
-o ../../../WorkingFolder/Stacks_2/tagsGenepop_20170322.genepop